In [1]:
%pip install keras -q
%pip install keras_cv -q
%pip install keras_core -q

%pip install tensorflow -q
%pip install numpy -q

%pip install wandb -q
%pip install ipywidgets -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from os import listdir, mkdir
from os.path import isdir, join

from tqdm.auto import tqdm
import xml.etree.ElementTree as ET

import tensorflow
from tensorflow.ragged import constant

import keras
from keras import Model
from keras.layers import InputLayer

import keras_cv
from keras_cv import bounding_box, visualization
from random import SystemRandom

from keras.utils import plot_model

import wandb
from wandb import init
from wandb.integration.keras import WandbMetricsLogger



2023-09-04 17:05:45.669881: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-04 17:05:45.969455: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-04 17:05:45.971625: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-04 17:05:47.694002: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Using TensorFlow backend


In [3]:
epoch: int = 15
batch_size: int = 4

learning_rate: float = 0.002

target_size_width: int = 640
target_size_height: int = 640


In [4]:
graph_model: bool = False

split_at: float = 0.15

global_clip_at: float = 10.0
boundary_format: str = 'xyxy'

In [5]:
labels: list = [
    'Dog',
    'Dog tag',
    'Collar'
]

labels_mapped: dict = dict(
    zip(
        range(len(labels)), 
        labels
    )
)

dataset_paths: dict = dict(
    {
        'images': '/home/jupyter/datasets/Tracking Madsen/Images',
        'annotations': '/home/jupyter/datasets/Tracking Madsen/Annotations'
    }
)

In [6]:
configuration: dict = {
    'target': {
        'width': target_size_width,
        'height': target_size_height
    },
    'model': {
        'epochs': epoch,
        'batch size': batch_size,
        'boundary format': boundary_format,
        'name': 'yolo'
    },
    'dataset':
    {
        'paths': dataset_paths,
        'labels': labels_mapped,
        'split': split_at
    },
    
}

In [7]:
init(
    project='Mjølner', 
    entity='designermadsen',
    save_code=True,
    config = configuration,
    sync_tensorboard = True,
    job_type='Training'
)

wandb: Currently logged in as: designermadsen. Use `wandb login --relogin` to force relogin


In [8]:
xml_files = sorted(
    [
        join(dataset_paths['annotations'], file_name)
        for file_name in listdir(dataset_paths['annotations'])
        if file_name.lower().endswith(".xml")
    ]
)

size_of_annotations = len(xml_files)

image_files = sorted(
    [
        join(dataset_paths['images'], file_name)
        for file_name in listdir(dataset_paths['images'])
        if file_name.lower().endswith(".jpg") or file_name.lower().endswith(".jpeg")
    ]
)

def parse_xml(
    xml_file
):
    global dataset_paths, labels_mapped
    tree = ET.parse(xml_file)
    root = tree.getroot()

    image_name = root.find("filename").text
    image_path = join(dataset_paths['images'], image_name)

    boxes = []
    classes = []

    for object in root.iter('object'):
        cls = object.find("name").text
        classes.append(cls)

        bounderies = object.find("bndbox")
        xmin: float = float(bounderies.find('xmin').text)
        ymin: float = float(bounderies.find('ymin').text)

        xmax: float = float(bounderies.find('xmax').text)
        ymax: float = float(bounderies.find('ymax').text)

        boxes.append([xmin, ymin, xmax, ymax])
    
    label_ids: list = [
        list(labels_mapped.keys())[list(labels_mapped.values()).index(cls)]
        for cls in classes
    ]
    
    return image_path, boxes, label_ids

image_paths = []
boundaries = []
classes = []

for file in tqdm(
    xml_files
):
    image_path, boxes, label_ids = parse_xml(
        file
    )
    
    image_paths.append(
        image_path
    )
    
    boundaries.append(
        boxes
    )
    
    classes.append(
        label_ids
    )

image_paths = constant(image_paths)
classes = constant(classes)
boundaries = constant(boundaries)

dataset = tensorflow.data.Dataset.from_tensor_slices((
    image_paths,
    classes,
    boundaries
))

def load_image(image_path):
    image = tensorflow.io.read_file(image_path)
    image = tensorflow.image.decode_jpeg(image, channels=3)
    return image

def load_dataset(
    image_path, 
    classes, 
    bounderies
):
    image = load_image(image_path)
    bounding_boxes = {
        "classes": tensorflow.cast(
            classes, 
            dtype=tensorflow.float32
        ),
        "boxes": bounderies
    }

    return {
        "images": tensorflow.cast(
            image, 
            dtype=tensorflow.float32
        ), 
        "bounding_boxes": bounding_boxes
    }

resize = keras.Sequential(
    layers=[
        keras_cv.layers.JitteredResize(
            target_size=(target_size_height, target_size_width),
            scale_factor=(0.5, 2.0),
            bounding_box_format=boundary_format
        ),
        
        keras_cv.layers.AutoContrast([0, 1])
    ]
)

number_of_validation: int = int(size_of_annotations * split_at)

validation_data = dataset.take(
    number_of_validation
)

training_data = dataset.skip(
    number_of_validation
)

training_data = training_data.map(
    load_dataset, 
    num_parallel_calls=tensorflow.data.AUTOTUNE
)

training_data = training_data.shuffle(batch_size * 4)

training_data = training_data.ragged_batch(
    batch_size, 
    drop_remainder=True
)

training_data = training_data.map(
    resize, 
    num_parallel_calls=tensorflow.data.AUTOTUNE
)


validation_data = validation_data.map(
    load_dataset, 
    num_parallel_calls=tensorflow.data.AUTOTUNE
)

validation_data = validation_data.shuffle(batch_size * 4)

validation_data = validation_data.ragged_batch(
    batch_size, 
    drop_remainder=True
)

validation_data = validation_data.map(
    resize, 
    num_parallel_calls=tensorflow.data.AUTOTUNE
)

def visualise_dataset(
        dataset_as_input, 
        value_range, 
        rows, 
        cols, 
        bounding_box_format
):
    global labels_mapped
    dataset_as_input = next(iter(dataset_as_input.take(1)))
    images, bounding_boxes = dataset_as_input["images"], dataset_as_input["bounding_boxes"]
    
    visualization.plot_bounding_box_gallery(
        images,
        value_range=value_range,
        rows=rows,
        cols=cols,
        y_true=bounding_boxes,
        scale=5,
        font_scale=0.5,
        bounding_box_format=bounding_box_format,
        class_mapping=labels_mapped
    )

visualise_dataset_before_run: bool = False

if visualise_dataset_before_run:
    visualise_dataset(
        training_data, 
        bounding_box_format=boundary_format, 
        value_range=(0, 255), 
        rows=2, 
        cols=2
    )

    visualise_dataset(
        validation_data, 
        bounding_box_format=boundary_format, 
        value_range=(0, 255), 
        rows=2, 
        cols=2
    )

def dict_to_tuple(
    inputs: dict
) -> tuple:
    return inputs['images'], inputs['bounding_boxes']

training_data = training_data.map(
    dict_to_tuple, 
    num_parallel_calls=tensorflow.data.AUTOTUNE
)

training_data = training_data.prefetch(
    tensorflow.data.AUTOTUNE
)

validation_data = validation_data.map(
    dict_to_tuple, 
    num_parallel_calls=tensorflow.data.AUTOTUNE
)

validation_data = validation_data.prefetch(
    tensorflow.data.AUTOTUNE
)

  0%|          | 0/1998 [00:00<?, ?it/s]

In [9]:
# Model Creation
backbone = keras_cv.models.YOLOV8Backbone.from_preset(
    'yolo_v8_l_backbone'
)

yolo = keras_cv.models.YOLOV8Detector(
    num_classes=len(labels_mapped),
    bounding_box_format=boundary_format,
    backbone=backbone,
    fpn_depth=2
)

In [10]:
optimizer = tensorflow.keras.optimizers.Adam(
    learning_rate=learning_rate,
    global_clipnorm=global_clip_at,
)

yolo.compile(
    optimizer=optimizer, 
    classification_loss="binary_crossentropy", 
    box_loss="ciou"
)

In [11]:
if graph_model:
    plot_model(
        yolo, 
        to_file='D:/Model/yolo.png', 
        show_shapes=True, 
        show_trainable=True
    )

In [12]:
histories: list = list()

In [ ]:
history = yolo.fit(
    training_data,
    validation_data=validation_data,
    epochs=epoch,
    use_multiprocessing=True,
    
    workers=8,
    
    callbacks=[
        WandbMetricsLogger()
    ]
)

histories.append(history)

Epoch 1/15
424/424 [==============================] - ETA: 0s - loss: 33.5694 - box_loss: 2.3042 - class_loss: 31.2652

In [ ]:
model_locations: str = '/home/jupyter/models/'
if not isdir(model_locations):
    mkdir(model_locations)

In [13]:
save_model_at: str = join(
    model_locations, 
    'yolo'
)

save_model_when_done: bool = True

save_to_wandb: bool = True

In [19]:
if not isdir(save_model_at):
    mkdir(save_model_at)

In [21]:
if save_model_when_done:
    yolo.save(
        filepath=save_model_at, 
        save_format='keras',
        overwrite=True
    )

if save_to_wandb and save_model_when_done:
    artifact = wandb.Artifact(
        name='mjoelner_vision_model', 
        type='model-keras'
    )

    artifact.add_dir(
        local_path=save_model_at
    )

    wandb.log_artifact(
        artifact
    )

INFO:tensorflow:Assets written to: /tmp/model/assets


INFO:tensorflow:Assets written to: /tmp/model/assets
wandb: Adding directory to artifact (/tmp/model)... Done. 1.1s


In [22]:
wandb.finish()

epoch/box_loss,▃▇█▁▁
epoch/class_loss,█▁▁▁▁
epoch/epoch,▁▃▅▆█
epoch/learning_rate,▁▁▁▁▁
epoch/loss,█▁▁▁▁
epoch/val_box_loss,█▆▆▁▁
epoch/val_class_loss,▆█▅▁▂
epoch/val_loss,█▇▆▁▂
epoch/box_loss,2.30197
epoch/class_loss,0.78561
epoch/epoch,4
